# Scraping The Surface of Mars

In [347]:
# Import Dependencies
import IPython
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd

In [200]:
# Mars URL page to be scraped
site_news_url = 'https://mars.nasa.gov/news/'

In [32]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [33]:
browser.visit(site_news_url)

In [9]:
html = browser.html
soup = bs(html, 'html.parser')

In [25]:
mars_subtags = soup.select_one('ul.item_list li.slide')

In [27]:
# Latest News Title
news_title = mars_subtags.find('div', class_='content_title').get_text()
news_title

'MOXIE Could Help Future Rockets Launch Off Mars'

In [28]:
# Paragraph Text
news_parag = mars_subtags.find('div', class_='article_teaser_body').get_text()
news_parag

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

In [208]:
site_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [209]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [210]:
browser.visit(site_image_url)

In [211]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [212]:
browser.links.find_by_partial_text('more info').click()

In [214]:
# Get featured image
new_url = browser.url

'https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA09320'

In [215]:
html = browser.html
soup = bs(html, 'html.parser')

In [293]:
mars_figure = soup.select_one('figure.lede a')['href']

In [294]:
mars_figure

'/spaceimages/images/largesize/PIA09320_hires.jpg'

In [348]:
large_image = 'https://www.jpl.nasa.gov' + mars_figure
IPython.display.HTML('<img src=https://www.jpl.nasa.gov' + mars_figure + ' width=55%>')